In [39]:
from supabase import create_client, Client
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
import arxiv
from datetime import datetime, timedelta
from sqlalchemy.exc import OperationalError
import psycopg2

In [40]:
def get_recent_arxiv_papers(weeks_ago=1, max_results=1000):
    # Calculate the date range
    today = datetime.now()
    start_date = (today - timedelta(weeks=weeks_ago)).strftime('%Y%m%d%H%M%S')
    end_date = today.strftime('%Y%m%d%H%M%S')

    # Create search query
    search_query = f'submittedDate:[{start_date} TO {end_date}]'

    # Initialize the arXiv API client
    search = arxiv.Search(
        query=search_query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    # Create lists to store the data
    papers_data = []

    # Iterate over the search results
    for result in search.results():
        paper = {
            'Title': result.title,
            'Abstract': result.summary,
            'Authors': ', '.join([author.name for author in result.authors]),
            'Published Date': result.published.strftime('%Y-%m-%d'),
            'Link': result.entry_id
        }
        papers_data.append(paper)

    # Convert to DataFrame
    return pd.DataFrame(papers_data)

In [52]:
load_dotenv('../.env')
print("Current Working Directory:", os.getcwd())

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
print(url)

supabase: Client = create_client(url, key)

FileNotFoundError: [Errno 2] No such file or directory

In [3]:

df = get_recent_arxiv_papers()

print('Data fetched successfully')

/var/folders/kd/j4v79_8x3vd7bqr4q1l0f5tc0000gn/T/ipykernel_58814/2931823615.py:21: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Data fetched successfully


In [44]:
data_dict = df.to_dict(orient='records')
# print(data_dict)
sample = data_dict[0]
print(sample)
response = supabase.table('arxiv_data').insert(data_dict).execute()
print(response)

print('Data inserted successfully')

{'Title': "The Guitar's Magnetic Field Revealed by Starlight Polarization", 'Abstract': 'The Guitar nebula surrounding PSR B2224+65 boasts a pulsar X-ray filament\nlikely aligned with the local magnetic field. We present new RoboPol stellar\npolarization data distributed along the line-of-sight to the pulsar. The\npolarizing effect of intervening magnetized dust allows us to extract a model\nfor the dust-weighted magnetic field. We detect a magnetic field angle\nconsistent with the filament if the pulsar is located in the more distant zone\nof its parallax-estimated distance range.', 'Authors': 'Jack T. Dinsmore, Roger W. Romani', 'Published Date': '2025-01-13', 'Link': 'http://arxiv.org/abs/2501.07577v1'}
data=[{'Title': "The Guitar's Magnetic Field Revealed by Starlight Polarization", 'id': 1002, 'Abstract': 'The Guitar nebula surrounding PSR B2224+65 boasts a pulsar X-ray filament\nlikely aligned with the local magnetic field. We present new RoboPol stellar\npolarization data distri

In [46]:
supabase.table('conversations').select('*').execute()


APIError: {'code': '42P01', 'details': None, 'hint': None, 'message': 'relation "public.conversations" does not exist'}